In [23]:
import numpy as np
import pandas as pd
import re



In [15]:
df = pd.read_csv("open_medicine/data/UpperGI.csv")

In [20]:
df = df.rename({"out": "text"}, axis=1)

In [21]:
df.head()

,Unnamed: 0,text,NA
0,out1,"Hospital Number: R1265623 , Hospital: Random N...",\nNA
1,out2,"Hospital Number: K2515095 , Hospital: Random N...",Nature of specimen:x9 stomach biopsy specimens...
2,out3,"Hospital Number: L7746099 , Hospital: Random N...",\ncharacter(0)
3,out4,"Hospital Number: J4131371 , Hospital: Random N...",
4,out5,"Hospital Number: S4637507 , Hospital: Random N...",Nature of specimen:x6 fundus biopsy specimens ...


In [24]:
def regex_indications(string):
    hospital_reg = r"\.*INDICATIONS FOR PROCEDURE:.*"
    line = re.findall(hospital_reg, string)[0]
    retrn_string =  line.replace(',',':').split(":")[1]
    if retrn_string[-1:] == "\r":
        retrn_string= retrn_string[:-1]
    if retrn_string[-8:] == "FINDINGS":
        return retrn_string[:-8]
    else:
        return retrn_string
    
df["indications"] = df['text'].apply(regex_indications)

In [25]:
def regex_extent(string):
    hospital_reg = r"\.*Extent of Exam:.*"
    line = re.findall(hospital_reg, string)[0]
    retrn_string =  line.replace(',',':').split(":")[1]
    if retrn_string[-1:] == "\r":
        return retrn_string[:-1]
    else:
        return retrn_string
df["extent_of_exam"] = df['text'].apply(regex_extent)

In [26]:
def regex_findings(string):
    hospital_reg = r"\.*FINDINGS:.*"
    line = re.findall(hospital_reg, string)[0][10:]
    return line
df["findings"] = df['text'].apply(regex_findings)

In [27]:
df.head()

,Unnamed: 0,text,NA,indications,extent_of_exam,findings
0,out1,"Hospital Number: R1265623 , Hospital: Random N...",\nNA,Ongoing reflux symptoms.,D1,Columnar lined oesophagus is present. The segm...
1,out2,"Hospital Number: K2515095 , Hospital: Random N...",Nature of specimen:x9 stomach biopsy specimens...,Endoscopic ultrasound findings,Oesophagus,There is an ulcer in the stomach which is supe...
2,out3,"Hospital Number: L7746099 , Hospital: Random N...",\ncharacter(0),Nausea and/or Vomiting Haematemesis or Melaen...,D1,LA Grade D oesophagitis. The oesopahgitis is ...
3,out4,"Hospital Number: J4131371 , Hospital: Random N...",,IDA,D2,There is a polyp in the body which is sessile ...
4,out5,"Hospital Number: S4637507 , Hospital: Random N...",Nature of specimen:x6 fundus biopsy specimens ...,Dysphagia/Odynophagia .,Pylorus,There is a stricture in the fundus which is Oe...


In [1]:
from transformers import pipeline

2023-03-07 16:15:41.889432: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 16:15:50.255516: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/opt/conda/lib
2023-03-07 16:15:50.257789: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [6]:
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 23.5 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=e24b8cd1a61156b1cf04cb476a92a97f4bad44e47290a8250efefa68969c5702
  Stored in directory: /root/.cache/pip/wheels/5b/e0/77/05245143a5b31f65af6a21f7afd3219e9fa4896f918af45677
Successfully built sacremoses


In [7]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")

model = AutoModelForCausalLM.from_pretrained("microsoft/biogpt")

In [8]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [11]:
generator("FINDINGS:")

[{'generated_text': 'FINDINGS: The results of this study suggest that the use of a combination of the two methods of analysis'}]

In [28]:
def tokenize_function(examples):
    findings = examples.tolist()
    tokenized_list = [tokenizer(text, padding="max_length", truncation=True) for text in list]
    return tokenized_list
#tokenizer(examples["text"], padding="max_length", truncation=True)

In [29]:
tokenized_findings = tokenize_function(df.findings)

TypeError: 'type' object is not iterable

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)